# **ÍNDICE FELIPE**

In [1]:
# cargue de librerias
import numpy as np
import pandas as pd

In [2]:
!pip install openpyxl
from openpyxl import Workbook
import openpyxl

Ahora procedemos a realizar el cargue de la base de datos del registro mercantil

In [4]:
BD=pd.read_excel("MATREM_CCC_ENE-AGO2022_V1(SEP_08_EEE3).xlsx",dtype={'CODIGO_CIIU':object, 'MATRICULA': object, 'NIT':object, 'Año de FECHA_RENOVACION':object, 'TELEFONO': object})
BD.info()
BD.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108454 entries, 0 to 108453
Data columns (total 38 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   MATRICULA                108454 non-null  object 
 1   NIT                      108454 non-null  object 
 2   ESTADO_MAT_REN           108454 non-null  object 
 3   MAT_REN                  108454 non-null  object 
 4   DESC_CATEGORIA           108454 non-null  object 
 5   ESTADO_ACTUAL            108454 non-null  object 
 6   SECTOR                   108454 non-null  object 
 7   RAZON_SOCIAL             108450 non-null  object 
 8   DESC_CIIU                108454 non-null  object 
 9   NOM_ENTE_JURIDICO        108454 non-null  object 
 10  DESC_TAMANO              108454 non-null  object 
 11  TAMANO957                108454 non-null  object 
 12  CIUDAD                   108452 non-null  object 
 13  Año de FECHA_RENOVACION  108454 non-null  object 
 14  FECH

,MATRICULA,NIT,ESTADO_MAT_REN,MAT_REN,DESC_CATEGORIA,ESTADO_ACTUAL,SECTOR,RAZON_SOCIAL,DESC_CIIU,NOM_ENTE_JURIDICO,...,UTILILIDAD_PERDIDA,UTILILIDAD_BRUTA,VENTAS,CONSECUTIVO_PROPIETARIO,NUM_CONSECUTIVO,CODIGO_CIIU,BARRIO,REP_LEGAL,AFILIADO,PORCENT_EXTRANJERO
0,60,890306499,A,Renovacion,PERSONA JURIDICA,ACTIVO,C,UNION PLASTICA S.A.S. ...,FABRICACIÓN DE ARTÍCULOS DE PLÁSTICO N.C.P. ...,Sociedad por Acciones Simplificada ...,...,6364618950,4.257448e+09,4.221629e+10,0.0,19,2229,Barrio Yumbo,JOSÉ GUILLERMO PARDO BORRERO,S,0.0
1,64,860026759,A,Renovacion,PERSONA JURIDICA,ACTIVO,C,CARTONES AMERICA S.A. CAME ...,FABRICACIÓN DE PAPEL Y CARTÓN ONDULADO (CORRUG...,Sociedad Anónima ...,...,79168212000,9.903336e+10,3.988043e+11,0.0,21,1702,Brisas De Los Alamos,JUAN ADALBERTO ZOLTAN ANDUJAR,S,30.0
2,159,4872952,A,Renovacion,PERSONA NATURAL,ACTIVO,G,JOJOA ROSERO JOSE FRANCISCO ...,COMERCIO AL POR MENOR EN ESTABLECIMIENTOS NO E...,Persona Natural ...,...,0,0.000000e+00,0.000000e+00,0.0,54,4711,La Campina,JOSE FRANCISCO JOJOA ROSERO,N,0.0
3,197,14447242,A,Renovacion,PERSONA NATURAL,ACTIVO,C,QUINTANA CORREA JOSE SAULO ...,FABRICACIÓN DE HOJAS DE MADERA PARA ENCHAPADO;...,Persona Natural ...,...,2351028,2.635103e+07,2.351028e+06,0.0,69,1620,Centenario,JOSE SAULO QUINTANA CORREA,N,0.0
4,217,2401807,A,Renovacion,PERSONA NATURAL,ACTIVO,H,RINCON CASTRO ROMULO DE JESUS ...,TRANSPORTE DE PASAJEROS ...,Persona Natural ...,...,277811736,2.778117e+08,8.500777e+08,0.0,75,4921,Santa Teresita,ROMULO DE JESUS RINCON CASTRO,N,NaN


# Medición de calidad de datos por dimensión

## Dimensión Exactitud

In [5]:
## ACTIVOS TOTALES
activos_total=len(BD['TOT_ACTIVOS'].dropna()) #Se borran las entradas que no tienen información
activos_cero=sum(BD['TOT_ACTIVOS']==0) #Se saca aparte un grupo con los Activos que son iguales a cero 
activos_fuera_rango=sum(BD['TOT_ACTIVOS']<0) #Se saca aparte un grupo con los activos negativos

#El objetivo es buscar qué tan bien están las caracterísitcas para el índice
#El cero lo sacamos porque no se está plenamente seguro de donde si se digitó o no la información 

metrica_activos_rango= 1 - (activos_fuera_rango/(activos_total-activos_cero))
metrica_activos_rango

1.0

In [44]:
## PASIVOS TOTALES

##Es correcto o incorrecto que los pasivos sean negativos.
pasivos_total=len(BD['TOT_PASIVO'].dropna())
pasivos_cero=sum(BD['TOT_PASIVO']==0) 
pasivos_fuera_rango=sum(BD['TOT_PASIVO']<0)

print(pasivos_fuera_rango)

metrica_pasivos_rango= 1 - (pasivos_fuera_rango/(pasivos_total-pasivos_cero))
metrica_pasivos_rango

9


0.9996909765142151

In [7]:
## VENTAS
ventas_total=len(BD['VENTAS'].dropna())
ventas_cero=sum(BD['VENTAS']==0)
ventas_fuera_rango=sum(BD['VENTAS']<0)

# print(ventas_total);print(ventas_cero);print(ventas_fuera_rango)

metrica_ventas_rango= 1 - (ventas_fuera_rango/(ventas_total-ventas_cero))
metrica_ventas_rango

0.9998906685617449

### Indicador de calidad - Exactitud

In [8]:
indicador_exactitud_minimo=min(metrica_activos_rango,metrica_pasivos_rango,metrica_ventas_rango)
indicador_exactitud_promedio=np.mean([metrica_activos_rango,metrica_pasivos_rango,metrica_ventas_rango])

print(f'El indicador de la dimensión EXACTITUD por medio del mínimo es: {np.round(indicador_exactitud_minimo,5)}')
print(f'El indicador de la dimensión EXACTITUD por medio del promedio es: {np.round(indicador_exactitud_promedio,5)}')

El indicador de la dimensión EXACTITUD por medio del mínimo es: 0.99989
El indicador de la dimensión EXACTITUD por medio del promedio es: 0.99994


## Dimensión Completitud

In [9]:
## ACTIVOS COMPLETITUD
registros_totales=len(BD) ## Cantidad de registros de la base de datos 
#Estamos buscando calcular el número de registros que tienen ACTIVOS nulo. A la cantidad de registros totales
#le restamos el número de registros que no son nulos y le sumamos los que tienen activos iguales a cero que también actuan como nulos
#Básicamente hicimos Registros Nulos = Registros Totales - Registros que no son nulos + Registros que tienen activos iguales a cero
activos_nulos=registros_totales-len(BD['TOT_ACTIVOS'].dropna())+activos_cero 

activos_completos= 1- (activos_nulos/registros_totales)
activos_completos

0.9973168347870986

In [10]:
## PASIVOS COMPLETITUD 
pasivos_nulos=registros_totales-len(BD['TOT_PASIVO'].dropna())

pasivos_completos= 1-(pasivos_nulos/registros_totales)
pasivos_completos

0.9999907795009866

In [11]:
## VENTAS (INGRESOS_ACT_ORDINARIA)
ventas_nulos=registros_totales - len(BD['VENTAS'].dropna()) + ventas_cero

ventas_completos= 1- (ventas_nulos/registros_totales)
ventas_completos

0.33734117690449406

In [12]:
## PATRIMONIO
patrimonio_nulos=registros_totales - len(BD['PATRIMONIO'].dropna())

patrimonio_completos= 1-(patrimonio_nulos/registros_totales)
patrimonio_completos


0.9999907795009866

In [13]:
## NIT
nit_nulos=registros_totales - len(BD['NIT'].dropna()) + sum(BD['NIT']==0)

nit_completos=1-(nit_nulos/registros_totales)
nit_completos

0.9978792852269165

In [14]:
## SECTOR
sector_nulos=registros_totales - len(BD['SECTOR'].dropna()) + sum(BD['SECTOR']==0)

sector_completos= 1- (sector_nulos/registros_totales)
sector_completos

1.0

In [15]:
## CIUDAD
ciudad_nulos=registros_totales - len(BD['CIUDAD'].dropna())

ciudad_completos=1 - (ciudad_nulos/registros_totales)
ciudad_completos

0.9999815590019732

In [16]:
## COMUNA
#Se evalúa una subbase con los registros que pertenecen a Cali
BD_cali=BD[BD['CIUDAD']=='Cali']
registros_tot_comunas=len(BD_cali)

comuna_nulos= registros_tot_comunas - len(BD_cali['COMUNA'].dropna())

comuna_completos= 1- (comuna_nulos/registros_tot_comunas)
comuna_completos

0.9517231484992263

In [17]:
#UTILIDAD PERDIDA (UTILIDAD OPERACIONAL)
#Cómo manejar el hecho que la variable tenga tantos registros iguales a cero, los manejamos como nulos?
utilidad_perdida_nulos=registros_totales - len(BD['UTILILIDAD_PERDIDA'].dropna())

#Formas de revisar la cantidad de ceros que tiene esta variable 
#sum(BD['UTILILIDAD_PERDIDA'] == 0)
#print((BD['UTILILIDAD_PERDIDA']==0).value_counts())


utilidad_perdida_completos = 1 - (utilidad_perdida_nulos/registros_totales)
utilidad_perdida_completos


1.0

In [18]:
## UTILIDAD BRUTA (RESULTADO_PERIODO)
utilidad_bruta_nulos= registros_totales - len(BD['UTILILIDAD_BRUTA'].dropna())

#Siempres es 1 menos el complemento (lo que está mal). El resultado nos da la proporción de la variable que está bien

utilidad_bruta_completos= 1- (utilidad_bruta_nulos/registros_totales)
utilidad_bruta_completos

0.9999907795009866

### Indicador de Calidad - Completitud 

In [19]:
indicador_completitud_minimo=min(activos_completos,pasivos_completos,patrimonio_completos,ventas_completos,
                                nit_completos,sector_completos,ciudad_completos,comuna_completos, utilidad_perdida_completos,
                                utilidad_bruta_completos)
indicador_completitud_promedio= np.mean([activos_completos,pasivos_completos,patrimonio_completos,ventas_completos,
                                nit_completos,sector_completos,ciudad_completos,comuna_completos, utilidad_perdida_completos,
                                utilidad_bruta_completos])

print(f'El indicador de la dimensión COMPLETITUD por medio del mínimo es: {np.round(indicador_completitud_minimo,5)}')
print(f'El indicador de la dimensión COMPLETITUD por medio del promedio es: {np.round(indicador_completitud_promedio,5)}')

El indicador de la dimensión COMPLETITUD por medio del mínimo es: 0.33734
El indicador de la dimensión COMPLETITUD por medio del promedio es: 0.92842


## Dimensión Consistencia

In [20]:
## Relación CIUDAD-COMUNA
#Cumplimiento de relaciones dentro de la Base de Datos. Por ejemplo, las comunas de Cali no pueden tomar el valor de Comuna 80.
#Las relaciones inter-relación es entre bases de datos
ciudad_comuna_error1=sum(BD_cali['COMUNA']=='Comuna 80')

ciudad_comuna_consistencia= 1 - (ciudad_comuna_error1/len(BD_cali))
ciudad_comuna_consistencia

0.9905311375956878

In [21]:
## Relación PATRIMONIO = ACTIVO - PASIVO

#Sacamos los registros donde hay información para las tres variables al tiempo
BD_patrimonio=BD[BD['PATRIMONIO'].notna() & BD['TOT_ACTIVOS'].notna() & BD['TOT_PASIVO'].notna()] ##Registros donde hay información para evaluar la ecuación contable

patrimonio_calc= BD_patrimonio['TOT_ACTIVOS'] - BD_patrimonio['TOT_PASIVO'] 

#Revisar si la forma en que calculamos la ecuación contable corresponde al dato que hay en la base

compara_patrimonio= BD_patrimonio['PATRIMONIO'] == patrimonio_calc

##Formas de revisar cuántos cumplen la condición de la ecuación contable
print(compara_patrimonio.value_counts())

print((BD['UTILILIDAD_PERDIDA']==0).value_counts())

patrimonio_consistencia = sum(compara_patrimonio)/len(patrimonio_calc)
patrimonio_consistencia

True     108439
False        14
dtype: int64
True     70601
False    37853
Name: UTILILIDAD_PERDIDA, dtype: int64


0.9998709118235549

### Indicador de calidad - Consistencia

In [22]:
indicador_consistencia_minimo=min(ciudad_comuna_consistencia,patrimonio_consistencia)
indicador_consistencia_promedio= np.mean([ciudad_comuna_consistencia,patrimonio_consistencia])

print(f'El indicador de la dimensión CONSISTENCIA por medio del mínimo es: {np.round(indicador_consistencia_minimo,5)}')
print(f'El indicador de la dimensión CONSISTENCIA por medio del promedio es: {np.round(indicador_consistencia_promedio,5)}')

El indicador de la dimensión CONSISTENCIA por medio del mínimo es: 0.99053
El indicador de la dimensión CONSISTENCIA por medio del promedio es: 0.9952


## Dimensión Unicidad

In [23]:
#Copia de la BD inicial
df=BD.copy()
df.rename(columns={'Año de FECHA_RENOVACION':'ULTIMO_ANO_RENOVADO',
                          'FECHA_MAT_REN':'FECHA_MATRICULA_RENOVACION_HORA',
                          'TOT_ACTIVOS':'ACTIVO_TOTAL',
                          'UTILILIDAD_PERDIDA': 'UTILILIDAD_OPERACIONAL',
                          'VENTAS' : 'INGRESOS_ACTIVIDAD_ORDINARIA',
                          'TOT_PASIVO': 'PASIVO_TOTAL',
                          'UTILILIDAD_BRUTA': 'RESULTADO_PERIODO',
                          'MAT_REN' : 'MATRICULA_RENOVACION'}, 
                 inplace=True)

In [24]:
# Se separa la fecha para facilitar el análisis
df[['FECHA_MATRICULA_RENOVACION','HORA_FECHA_MAT_REN']] =  df.FECHA_MATRICULA_RENOVACION_HORA.str.split(' ', n=1, expand=True)

In [25]:
# Se separa día, mes y año para facilitar el análisis
df[['DIA','MES', 'ANO']] =  df.FECHA_MATRICULA_RENOVACION.str.split('/', n=3, expand=True)
#Se convierte la variable FECHA_MATRICULA_RENOVACION_HORA en datetime para facilitar manupulación de los datos
df.FECHA_MATRICULA_RENOVACION_HORA = pd.to_datetime(df.FECHA_MATRICULA_RENOVACION_HORA)

In [26]:
#Se convierte la variable FECHA_MATRICULA en datetime para facilitar manupulación de los datos
df.FECHA_MATRICULA = pd.to_datetime(df.FECHA_MATRICULA)

In [27]:
#Se organiza el DataFrame según la fecha del movimiento registrado. Esto facilitará las particiones que se harán de la base de datos
df=df.sort_values('FECHA_MATRICULA_RENOVACION_HORA')
print(df.info())
df.head(2)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108454 entries, 81901 to 63007
Data columns (total 43 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   MATRICULA                        108454 non-null  object        
 1   NIT                              108454 non-null  object        
 2   ESTADO_MAT_REN                   108454 non-null  object        
 3   MATRICULA_RENOVACION             108454 non-null  object        
 4   DESC_CATEGORIA                   108454 non-null  object        
 5   ESTADO_ACTUAL                    108454 non-null  object        
 6   SECTOR                           108454 non-null  object        
 7   RAZON_SOCIAL                     108450 non-null  object        
 8   DESC_CIIU                        108454 non-null  object        
 9   NOM_ENTE_JURIDICO                108454 non-null  object        
 10  DESC_TAMANO                      108454 n

,MATRICULA,NIT,ESTADO_MAT_REN,MATRICULA_RENOVACION,DESC_CATEGORIA,ESTADO_ACTUAL,SECTOR,RAZON_SOCIAL,DESC_CIIU,NOM_ENTE_JURIDICO,...,CODIGO_CIIU,BARRIO,REP_LEGAL,AFILIADO,PORCENT_EXTRANJERO,FECHA_MATRICULA_RENOVACION,HORA_FECHA_MAT_REN,DIA,MES,ANO
81901,1140755,1193226317,A,Renovacion,PERSONA NATURAL,CANCELADO,K,ELAGO BOTOTO LUZ ENITH ...,ACTIVIDADES DE AGENTES Y CORREDORES DE SEGUROS...,Persona Natural ...,...,6621,Jorge Zawadsky,LUZ ENITH ELAGO BOTOTO,N,NaN,1/02/2022,12:00:00 a.m.,1,02,2022
81867,1140731,1067462430,A,Matricula,PERSONA NATURAL,ACTIVO,G,PAJA URMENDIZ BREIDY ALEXANDER ...,COMERCIO AL POR MENOR DE PRODUCTOS FARMACÉUTIC...,Persona Natural ...,...,4773,Urbanizacion Caney,BREIDY ALEXANDER PAJA URMENDIZ,N,NaN,1/02/2022,12:00:00 a.m.,1,02,2022


In [30]:
df.groupby('ESTADO_MAT_REN').nunique()

,MATRICULA,NIT,MATRICULA_RENOVACION,DESC_CATEGORIA,ESTADO_ACTUAL,SECTOR,RAZON_SOCIAL,DESC_CIIU,NOM_ENTE_JURIDICO,DESC_TAMANO,...,CODIGO_CIIU,BARRIO,REP_LEGAL,AFILIADO,PORCENT_EXTRANJERO,FECHA_MATRICULA_RENOVACION,HORA_FECHA_MAT_REN,DIA,MES,ANO
ESTADO_MAT_REN,,,,,,,,,,,,,,,,,,,,,
A,93243,93089,2,2,2,20,93163,1120,11,5,...,466,496,87567,2,85,240,33759,31,8,1
D,187,187,2,2,2,17,187,101,4,4,...,98,59,182,2,2,97,22,30,8,1
R,49,49,1,2,2,14,49,37,5,4,...,37,31,48,2,2,36,14,22,7,1


In [31]:
# Para obtener los duplicados EXCEPTO la primera observación revisando todas las columnas 
duplicados = df[df.duplicated()]
print('El número de registros duplicados con todas las columnas iguales son:')
print (duplicados.MATRICULA.count())

El número de registros duplicados con todas las columnas iguales son:
0


In [32]:
# Antes de partir la base de datos. Se hace un conteo para saber cuantas Matriculas y Renovaciones se tienen en la base sin eliminar duplicados
df.MATRICULA_RENOVACION.value_counts()

Renovacion    93464
Matricula     14990
Name: MATRICULA_RENOVACION, dtype: int64

In [33]:
# Ahora es necesario partir la base de datos en dos para hacer la correspondiente limpieza. Se requieren tratar las empresas Matriculadas y Renovadas diferenciadas por su fecha de registro
df_matricula = df.loc[df['MATRICULA_RENOVACION'].isin(['Matricula'])]
df_renovacion = df.loc[df['MATRICULA_RENOVACION'].isin(['Renovacion'])]
print('El número de registros en la BD para Matriculas es:', df_matricula.MATRICULA_RENOVACION.value_counts(), sep='\n')
print('El número de registros en la BD para Renovaciones:', df_renovacion.MATRICULA_RENOVACION.value_counts(), sep='\n')

El número de registros en la BD para Matriculas es:
Matricula    14990
Name: MATRICULA_RENOVACION, dtype: int64
El número de registros en la BD para Renovaciones:
Renovacion    93464
Name: MATRICULA_RENOVACION, dtype: int64


In [34]:
# Aqui DEBE IR EL CODIGO DESPUES DE LIMPIAR EL TEMA DE RENOVACIONES MAS VIEJAS QUE LAS MATRICULAS. SE DEBEN ELIMINAR LAS RENOVACIONES CON FECHAS ANTERIORES A LAS DE LAS MATRICULAS porque esto es un indicador que la empresa hizo un translado de domicilio y se trajo todo su historial a la CCC
# Se convierte la variable FECHA_MATRICULA_RENOVACION_HORA en datetime para facilitar manupulación de los datos
df_renovacion.FECHA_MATRICULA_RENOVACION = pd.to_datetime(df_renovacion.FECHA_MATRICULA_RENOVACION)
df_renovacion = df_renovacion.loc[df_renovacion['FECHA_MATRICULA']<=df_renovacion['FECHA_MATRICULA_RENOVACION']]

C:\Users\ladoming\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '13/01/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\ladoming\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '14/01/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\ladoming\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '15/01/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\ladoming\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '16/01/2022' in DD/MM/YYYY format. Provide form

C:\Users\ladoming\AppData\Local\Temp\ipykernel_20028\264757128.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_renovacion.FECHA_MATRICULA_RENOVACION = pd.to_datetime(df_renovacion.FECHA_MATRICULA_RENOVACION)


In [35]:
# Se organizan las bases según la fecha en la cual se hizo el movimiento corresspondiente a Matricual o Renovación para posterior limpieza. Vale aclarar que esta el ordenamiento es descendente, es decir, se comienza desde YYYY/01/01 00:00 hasta YYYY/12/31 23:59 (de enero a diciembre)
df_matricula=df_matricula.sort_values('FECHA_MATRICULA_RENOVACION_HORA')
df_renovacion=df_renovacion.sort_values('FECHA_MATRICULA_RENOVACION_HORA')
df_renovacion.head(2)

,MATRICULA,NIT,ESTADO_MAT_REN,MATRICULA_RENOVACION,DESC_CATEGORIA,ESTADO_ACTUAL,SECTOR,RAZON_SOCIAL,DESC_CIIU,NOM_ENTE_JURIDICO,...,CODIGO_CIIU,BARRIO,REP_LEGAL,AFILIADO,PORCENT_EXTRANJERO,FECHA_MATRICULA_RENOVACION,HORA_FECHA_MAT_REN,DIA,MES,ANO
81901,1140755,1193226317,A,Renovacion,PERSONA NATURAL,CANCELADO,K,ELAGO BOTOTO LUZ ENITH ...,ACTIVIDADES DE AGENTES Y CORREDORES DE SEGUROS...,Persona Natural ...,...,6621,Jorge Zawadsky,LUZ ENITH ELAGO BOTOTO,N,NaN,2022-01-02,12:00:00 a.m.,1,02,2022
81899,1140754,1144125962,A,Renovacion,PERSONA NATURAL,ACTIVO,S,ALVAREZ VALDES SANDRA LADY ...,PELUQUERÍA Y OTROS TRATAMIENTOS DE BELLEZA ...,Persona Natural ...,...,9602,NaN,SANDRA LADY ALVAREZ VALDES,N,NaN,2022-01-02,12:00:00 a.m.,1,02,2022


In [36]:
# Se procede a eliminar los duplicados en las dos bases y la idea es dejar el último movimiento registrado. Esto se debe gracias a que se identificó que algunas empresas registran algun movimiento y por algun motivo este es devuelto, en la mayoría de los casos la corrección sugerida es hecha en el plazo para mantener la fecha inicial del movimiento, sin embargo, cuando la empresa deja pasar este plazo máximo se debe generar un nuevo movimiento lo cual genera duplicados en la información
df2_matricula=df_matricula.sort_values('FECHA_MATRICULA_RENOVACION_HORA').drop_duplicates('MATRICULA', keep='last')
df2_renovacion=df_renovacion.sort_values('FECHA_MATRICULA_RENOVACION_HORA').drop_duplicates('MATRICULA', keep='last')

In [37]:
# Una vez eliminados lo duplicados se cuenta el número de resgitros 
print('El número de registros en la BD para Matriculas después de eliminar duplicados es:', df2_matricula.MATRICULA_RENOVACION.value_counts(), sep='\n')
print('El número de registros en la BD para Renovaciones después de eliminar duplicados es:', df2_renovacion.MATRICULA_RENOVACION.value_counts(), sep='\n')

El número de registros en la BD para Matriculas después de eliminar duplicados es:
Matricula    14988
Name: MATRICULA_RENOVACION, dtype: int64
El número de registros en la BD para Renovaciones después de eliminar duplicados es:
Renovacion    93248
Name: MATRICULA_RENOVACION, dtype: int64


In [38]:
# Se procede a unir las bases de datos para la posterior limpieza en conjunto
df2 = pd.concat([df2_matricula, df2_renovacion])
df2 = df2.sort_values('MATRICULA_RENOVACION', ascending=True)
print('El número de registros en la BD unida es:', df2.MATRICULA_RENOVACION.value_counts(), sep='\n')

El número de registros en la BD unida es:
Renovacion    93248
Matricula     14988
Name: MATRICULA_RENOVACION, dtype: int64


In [39]:
# Para eliminar duplicados se debe tener en cuenta que en Matricula nos interesa dejar el primer registro que tengamos en la base pues por cada matricula se genera una renovación
df3 = df2.sort_values('MATRICULA_RENOVACION', ascending=True).drop_duplicates('MATRICULA', keep='first')
print('El número de registros en la BD unida sin duplicados es:', df3.MATRICULA_RENOVACION.value_counts(), sep='\n')

El número de registros en la BD unida sin duplicados es:
Renovacion    78264
Matricula     14988
Name: MATRICULA_RENOVACION, dtype: int64


### Indicador de calidad - Unicidad

In [40]:
## Calculo indicador duplicado

porcentaje_no_duplicados= (len(df3)/len(df))
porcentaje_no_duplicados

#

0.8598299739981928

# Cálculo de indicador de calidad agregado

In [41]:
IQA_min=(0.05*indicador_exactitud_minimo) + (0.3*indicador_completitud_minimo) + (0.05*indicador_consistencia_minimo) + (0.6*porcentaje_no_duplicados)
IQA_promedio=(0.05*indicador_exactitud_promedio) + (0.3*indicador_completitud_promedio) + (0.05*indicador_consistencia_promedio) + (0.6*porcentaje_no_duplicados)

##Minimo
print(f'Valor del IQA tomando como agregación el mínimo: \n')
print(f'Dimensión exactitud: {np.round(indicador_exactitud_minimo,5)}')
print(f'Dimensión completitud: {np.round(indicador_completitud_minimo,5)}')
print(f'Dimensión consistencia: {np.round(indicador_consistencia_minimo,5)}')
print(f'Dimensión unicidad: {np.round(porcentaje_no_duplicados,5)} \n')

print(f'El IQA obtenido por medio del método del mínimo es : {np.round(IQA_min,5)} \n')

##Promedio
print(f'Valor del IQA tomando como agregación el promedio: \n')
print(f'Dimensión exactitud: {np.round(indicador_exactitud_promedio,5)}')
print(f'Dimensión completitud: {np.round(indicador_completitud_promedio,5)}')
print(f'Dimensión consistencia: {np.round(indicador_consistencia_promedio,5)}')
print(f'Dimensión unicidad: {np.round(porcentaje_no_duplicados,5)} \n')

print(f'El IQA obtenido por medio del método del promedio es: {np.round(IQA_promedio,5)}')

Valor del IQA tomando como agregación el mínimo: 

Dimensión exactitud: 0.99989
Dimensión completitud: 0.33734
Dimensión consistencia: 0.99053
Dimensión unicidad: 0.85983 

El IQA obtenido por medio del método del mínimo es : 0.71662 

Valor del IQA tomando como agregación el promedio: 

Dimensión exactitud: 0.99994
Dimensión completitud: 0.92842
Dimensión consistencia: 0.9952
Dimensión unicidad: 0.85983 

El IQA obtenido por medio del método del promedio es: 0.89418


# Cálculo con pesos proporcionales 

In [42]:
IQA_min=(0.25*indicador_exactitud_minimo) + (0.25*indicador_completitud_minimo) + (0.25*indicador_consistencia_minimo) + (0.25*porcentaje_no_duplicados)
IQA_promedio=(0.25*indicador_exactitud_promedio) + (0.25*indicador_completitud_promedio) + (0.25*indicador_consistencia_promedio) + (0.25*porcentaje_no_duplicados)

##Minimo
print(f'Valor del IQA tomando como agregación el mínimo: \n')
print(f'Dimensión exactitud: {np.round(indicador_exactitud_minimo,5)}')
print(f'Dimensión completitud: {np.round(indicador_completitud_minimo,5)}')
print(f'Dimensión consistencia: {np.round(indicador_consistencia_minimo,5)}')
print(f'Dimensión unicidad: {np.round(porcentaje_no_duplicados,5)} \n')

print(f'El IQA obtenido por medio del método del mínimo es : {np.round(IQA_min,5)} \n')

##Promedio
print(f'Valor del IQA tomando como agregación el promedio: \n')
print(f'Dimensión exactitud: {np.round(indicador_exactitud_promedio,5)}')
print(f'Dimensión completitud: {np.round(indicador_completitud_promedio,5)}')
print(f'Dimensión consistencia: {np.round(indicador_consistencia_promedio,5)}')
print(f'Dimensión unicidad: {np.round(porcentaje_no_duplicados,5)} \n')

print(f'El IQA obtenido por medio del método del promedio es: {np.round(IQA_promedio,5)}')

Valor del IQA tomando como agregación el mínimo: 

Dimensión exactitud: 0.99989
Dimensión completitud: 0.33734
Dimensión consistencia: 0.99053
Dimensión unicidad: 0.85983 

El IQA obtenido por medio del método del mínimo es : 0.7969 

Valor del IQA tomando como agregación el promedio: 

Dimensión exactitud: 0.99994
Dimensión completitud: 0.92842
Dimensión consistencia: 0.9952
Dimensión unicidad: 0.85983 

El IQA obtenido por medio del método del promedio es: 0.94585


# **ÍNDICE JLC**